## ANÁLISIS DE ELASTICIDAD DEL ESPACIO PARA ÉXITO POR SUBZONA Y FORMATO

In [1]:
import numpy as np 
import scipy.optimize as opt
from scipy.optimize import minimize
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import math
import plotly.graph_objects as go
from scipy.optimize import Bounds

## LECTURA E IDENTIFICACIÓN DE DATOS 

In [2]:
full_data = pd.read_csv('Elasticidad del espacio/full_data_format.csv', sep=',')
full_data_2 = pd.read_csv('Elasticidad del espacio/full_data_format.csv', sep=',')

**DEFINIMOS LAS BASES DE DATOS PARA CADA TIPO DE FORMATO**

In [3]:
#BASES DE DATOS POR TIPO DE FORMATO
hiper = full_data[full_data.FormatDesc == 'HIPERMERCADOS']
supermercado = full_data[full_data.FormatDesc == 'SUPERMERCADO']
vecino = full_data[full_data.FormatDesc == 'VECINO']

In [4]:
#BASE DE DATOS CON LA INFORMACIÓN POR DEPENDENCIA
data_loc = full_data.iloc[:,[0,1,13,16,14,15,3]]
data_loc = data_loc.drop_duplicates()

## ASIGNACIÓN DE SERIES POR SUBZONA Y MACROCATEGORÍA

In [6]:
#OBTENEMOS UNA LISTA DE TUPLAS CON EL CÓDIGO Y LA DESCRIPCIÓN DE LA MACRO 
list_1 = full_data['DESC_MACRO'].unique()
list_2 = full_data['MACRO'].unique()

mac_comb = []
i = 0
for mac in hiper['MACRO'].unique():
    if mac == list_2[i]:
        mac_comb.append((mac, list_1[i]))
    i += 1


In [7]:
#CONVERTIMOS LA LISTA DE TUPLAS EN UN DATAFRAME
macro_df = pd.DataFrame(mac_comb, columns=['MACRO_ID', 'DESC_MAC']) 

In [8]:
#DEFINIMOS LA FUNCIÓN PARA CALCULAR LAS SERIES
def series(sub, macro, df):
    serie = df[['N° DEP.', 'DEPENDENCIA', 
                'MACRO', 'DESC_MACRO', 'CUERPOS FINALES', 'Vta_Und',
               'VentaSinIva']].loc[(df['SubzoneDesc'] == sub) & (df['DESC_MACRO'] == macro)]
    
    return serie

**ANÁLISIS FORMATO HIPERMERCADO**

In [9]:
#CALCULAMOS TODAS LAS POSIBLES COMBINACIONES ENTRE LAS SUBZONAS Y LAS MACROS PARA EL FORMATO HIPERMERCADO 
#Y LO GUARDAMOS EN UNA LISTA DE TUPLAS
combinations = []
for sub in hiper['SubzoneDesc'].unique():
    for macro in hiper['DESC_MACRO'].unique():
        combinations.append((sub, macro))


In [10]:
#DEFINIMOS LA FUNCIÓN DE VENTAS Y EL ERROR CUADRÁTICO 
def sales(a,x):
    
    return a[0]*(1-np.exp(-a[1]*x))

def q_err(a,x,y):
    
     return sum((sales(a,x) - y)**2)


In [11]:
#PARA CADA COMBINACIÓN DE SUBZONA Y MACRO OBTENEMOS UNA SERIE Y LE APLICAMOS LA OPTIMIZACIÓN
#LOS PARÁMETROS ÓPTIMOS CALCULADOS LOS GUARDAMOS EN UNA LISTA DE TUPLAS
#ESTA PRIMERA OPTIMIZACIÓN ES TENIENDO EN CUENTA LA VENTA EN PESOS

parameters = []

for comb in combinations:

    data = series(comb[0], comb[1], hiper)
    
    if len(data['CUERPOS FINALES'].unique()) > 2:
        
        x1 = data['CUERPOS FINALES']
        y1 = data['VentaSinIva']
        x0 = [1,1]
        max_1 = data['VentaSinIva'].max()
        bounds = Bounds([0, 0.11], [max_1, 2.30])
        
        sol = minimize(q_err, x0, args=(x1, y1), bounds= bounds)
        
        c_1 = sol.x[0]
        a_1 = sol.x[1]
        
        parameters.append((comb[0], comb[1], c_1, a_1))

In [12]:
#PARA CADA COMBINACIÓN DE SUBZONA Y MACRO OBTENEMOS UNA SERIE Y LE APLICAMOS LA OPTIMIZACIÓN
#LOS PARÁMETROS ÓPTIMOS CALCULADOS LOS GUARDAMOS EN UNA LISTA DE TUPLAS
#ESTA SEGUNDA OPTIMIZACIÓN ES TENIENDO EN CUENTA LA VENTA EN UNIDADES

parameters_1 = []

for comb in combinations:

    data = series(comb[0], comb[1], hiper)
    
    if len(data['CUERPOS FINALES'].unique()) > 2:
        
        x1 = data['CUERPOS FINALES']
        y1 = data['Vta_Und']
        x0 = [1,1]
        max_1 = data['Vta_Und'].max()
        bounds = Bounds([0, 0.11], [max_1, 2.30])
        
        sol = minimize(q_err, x0, args=(x1, y1), bounds= bounds)
        
        c_1 = sol.x[0]
        a_1 = sol.x[1]
        
        parameters_1.append((comb[0], comb[1], c_1, a_1))

In [13]:
#CONVERTIMOS LAS LISTAS DE TUPLAS DE LA OPTIMIZACIÓN EN UN DATAFRAME
data_hiper_1 = pd.DataFrame(parameters_1, columns= ['SUBZONA', 'DESC_MACRO', 'Venta_Maxima_Und', 'Alpha_Und'])
data_hiper_2 = pd.DataFrame(parameters, columns= ['SUBZONA', 'DESC_MACRO', 'Venta_Maxima', 'Alpha'])

In [14]:
#OBTENEMOS UN ÚNICO DATAFRAME CON LOS PARÁMETROS ÓPTIMOS PARA EL FORMATO HIPERMERCADO
elasticity_hiper = pd.concat([data_hiper_1,data_hiper_2], axis=1)
elasticity_hiper = elasticity_hiper.iloc[:,[0,1,2,3,6,7]]
elasticity_hiper['FORMATO'] = 'HIPERMERCADOS'

**ANÁLSISI FORMATO SUPERMERCADO**

In [15]:
#CALCULAMOS TODAS LAS POSIBLES COMBINACIONES ENTRE LAS SUBZONAS Y LAS MACROS PARA EL FORMATO SUPERMERCADO 
#Y LO GUARDAMOS EN UNA LISTA DE TUPLAS
combinations_super = []
for sub in supermercado['SubzoneDesc'].unique():
    for macro in supermercado['DESC_MACRO'].unique():
        combinations_super.append((sub, macro))

In [16]:
#PARA CADA COMBINACIÓN DE SUBZONA Y MACRO OBTENEMOS UNA SERIE Y LE APLICAMOS LA OPTIMIZACIÓN
#LOS PARÁMETROS ÓPTIMOS CALCULADOS LOS GUARDAMOS EN UNA LISTA DE TUPLAS
#ESTA PRIMERA OPTIMIZACIÓN ES TENIENDO EN CUENTA LA VENTA EN PESOS

parameters_super = []

for comb in combinations_super:

    data = series(comb[0], comb[1], supermercado)
    
    if len(data['CUERPOS FINALES'].unique()) > 2:
        
        x1 = data['CUERPOS FINALES']
        y1 = data['VentaSinIva']
        x0 = [1,1]
        max_1 = data['VentaSinIva'].max()
        bounds = Bounds([0, 0.11], [max_1, 2.30])
        
        sol = minimize(q_err, x0, args=(x1, y1), bounds= bounds)
        
        c_1 = sol.x[0]
        a_1 = sol.x[1]
        
        parameters_super.append((comb[0], comb[1], c_1, a_1))

In [17]:
#PARA CADA COMBINACIÓN DE SUBZONA Y MACRO OBTENEMOS UNA SERIE Y LE APLICAMOS LA OPTIMIZACIÓN
#LOS PARÁMETROS ÓPTIMOS CALCULADOS LOS GUARDAMOS EN UNA LISTA DE TUPLAS
#ESTA SEGUNDA OPTIMIZACIÓN ES TENIENDO EN CUENTA LA VENTA EN UNIDADES

parameters_super_1 = []

for comb in combinations_super:

    data = series(comb[0], comb[1], supermercado)
    
    if len(data['CUERPOS FINALES'].unique()) > 2:
        
        x1 = data['CUERPOS FINALES']
        y1 = data['Vta_Und']
        x0 = [1,1]
        max_1 = data['Vta_Und'].max()
        bounds = Bounds([0, 0.11], [max_1, 2.30])
        
        sol = minimize(q_err, x0, args=(x1, y1), bounds= bounds)
        
        c_1 = sol.x[0]
        a_1 = sol.x[1]
        
        parameters_super_1.append((comb[0], comb[1], c_1, a_1))

In [18]:
#CONVERTIMOS LAS LISTAS DE TUPLAS DE LA OPTIMIZACIÓN EN UN DATAFRAME
data_super_1 = pd.DataFrame(parameters_super_1, columns= ['SUBZONA', 'DESC_MACRO', 'Venta_Maxima_Und', 'Alpha_Und'])
data_super_2 = pd.DataFrame(parameters_super, columns= ['SUBZONA', 'DESC_MACRO', 'Venta_Maxima', 'Alpha'])

In [19]:
#OBTENEMOS UN ÚNICO DATAFRAME CON LOS PARÁMETROS ÓPTIMOS PARA EL FORMATO SUPERMERCADO
elasticity_super = pd.concat([data_super_1,data_super_2], axis=1)
elasticity_super = elasticity_super.iloc[:,[0,1,2,3,6,7]]
elasticity_super['FORMATO'] = 'SUPERMERCADOS'

**ANÁLISIS FORMATO VECINO**

In [20]:
#CALCULAMOS TODAS LAS POSIBLES COMBINACIONES ENTRE LAS SUBZONAS Y LAS MACROS PARA EL FORMATO VECINO 
#Y LO GUARDAMOS EN UNA LISTA DE TUPLAS

combinations_vecino = []
for sub in vecino['SubzoneDesc'].unique():
    for macro in vecino['DESC_MACRO'].unique():
        combinations_vecino.append((sub, macro))

In [21]:
#PARA CADA COMBINACIÓN DE SUBZONA Y MACRO OBTENEMOS UNA SERIE Y LE APLICAMOS LA OPTIMIZACIÓN
#LOS PARÁMETROS ÓPTIMOS CALCULADOS LOS GUARDAMOS EN UNA LISTA DE TUPLAS
#ESTA PRIMERA OPTIMIZACIÓN ES TENIENDO EN CUENTA LA VENTA EN PESOS

parameters_vecino = []

for comb in combinations_vecino:

    data = series(comb[0], comb[1], vecino)
    
    if len(data['CUERPOS FINALES'].unique()) > 2:
        
        x1 = data['CUERPOS FINALES']
        y1 = data['VentaSinIva']
        x0 = [1,1]
        max_1 = data['VentaSinIva'].max()
        bounds = Bounds([0, 0.11], [max_1, 2.30])
        
        sol = minimize(q_err, x0, args=(x1, y1), bounds= bounds)
        
        c_1 = sol.x[0]
        a_1 = sol.x[1]
        
        parameters_vecino.append((comb[0], comb[1], c_1, a_1))

In [22]:
#PARA CADA COMBINACIÓN DE SUBZONA Y MACRO OBTENEMOS UNA SERIE Y LE APLICAMOS LA OPTIMIZACIÓN
#LOS PARÁMETROS ÓPTIMOS CALCULADOS LOS GUARDAMOS EN UNA LISTA DE TUPLAS
#ESTA SEGUNDA OPTIMIZACIÓN ES TENIENDO EN CUENTA LA VENTA EN UNIDADES

parameters_vecino_1 = []

for comb in combinations_vecino:

    data = series(comb[0], comb[1], vecino)
    
    if len(data['CUERPOS FINALES'].unique()) > 2:
        
        x1 = data['CUERPOS FINALES']
        y1 = data['Vta_Und']
        x0 = [1,1]
        max_1 = data['Vta_Und'].max()
        bounds = Bounds([0, 0.11], [max_1, 2.30])
        
        sol = minimize(q_err, x0, args=(x1, y1), bounds= bounds)
        
        c_1 = sol.x[0]
        a_1 = sol.x[1]
        
        parameters_vecino_1.append((comb[0], comb[1], c_1, a_1))

In [23]:
#CONVERTIMOS LAS LISTAS DE TUPLAS DE LA OPTIMIZACIÓN EN UN DATAFRAME
data_vecino_1 = pd.DataFrame(parameters_vecino_1, columns= ['SUBZONA', 'DESC_MACRO', 'Venta_Maxima_Und', 'Alpha_Und'])
data_vecino_2 = pd.DataFrame(parameters_vecino, columns= ['SUBZONA', 'DESC_MACRO', 'Venta_Maxima', 'Alpha'])

In [24]:
#OBTENEMOS UN ÚNICO DATAFRAME CON LOS PARÁMETROS ÓPTIMOS PARA EL FORMATO SUPERMERCADO
elasticity_vecino = pd.concat([data_vecino_1,data_vecino_2], axis=1)
elasticity_vecino = elasticity_vecino.iloc[:,[0,1,2,3,6,7]]
elasticity_vecino['FORMATO'] = 'VECINO'

In [25]:
#CONCATENAMOS LOS DATAFRAMES DE CADA TIPO DE FORMATO Y OBTENEMOS UN ÚNICO DATAFRAME CON LOS PARÁMETROS ÓPTIMOS
elasticity = pd.concat([elasticity_hiper, elasticity_super], axis=0)
elasticity = elasticity.reset_index(drop=True)
elasticity = pd.concat([elasticity, elasticity_vecino], axis=0)
elasticity = elasticity.reset_index(drop=True)

In [26]:
#HACEMOS UN JOIN CON EL DATAFRAME 'macro_df' PARA AGREGAR A NUESTRA BASE DE DATOS LOS CÓDIGOS DE CADA MACRO
elasticity = pd.merge(left=elasticity, right=macro_df, how='left', left_on=['DESC_MACRO'], right_on=['DESC_MAC'])

In [27]:
#HACEMOS UN JOIN CON EL DATAFRAME 'data_loc' PARA AGREGAR A LA BASE DE DATOS LA INFORMACIÓN POR DEPENDENCIA
elasticity = pd.merge(left= data_loc, right= elasticity, how='left', left_on=['SubzoneDesc', 'FormatDesc', 'MACRO'], 
                      right_on=['SUBZONA', 'FORMATO', 'MACRO_ID'])

In [28]:
#NOS ASEGURAMOS QUE NO QUEDEN DATOS FALTANTES EN LA BASE DE DATOS Y FILTRAMOS LAS COLUMNAS DE INTERÉS
elasticity = elasticity.dropna()
elasticity = elasticity.iloc[:,[5,3,2,4,0,1,6,8,9,10,11,12]]

In [30]:
#elasticity.to_excel(r'C:/Users/jmcastellanos/Documents/Mateo Castellanos/Proyecto Elásticidad de Espacios/Datos/elasticity_data_dep_2.xlsx', index=False)

## ANÁLISIS DE LAS DISTRIBUCIONES ACTUALES DE CADA UNA DE LAS DEPENDENCIAS

**LECTURA DE DATOS**

In [31]:
elasticity_data = pd.read_excel('Elasticidad del espacio/elasticity_data_dep_2.xlsx')
bodies = pd.read_excel('Elasticidad del espacio/CUERPOS.xlsx')

In [34]:
#HACEMOS UN JOIN CON EL DATAFRAME 'bodies' PARA AGREGAR INFORMACIÓN DE LOS MUNDOS Y LOS CUERPOS ACTUALES
mundo = pd.merge(left=bodies, right= elasticity_data, how='left', left_on=['LOCID','LOC_DESC','COD_MAC'],
                right_on=['N° DEP.','DEPENDENCIA', 'MACRO'])

In [35]:
#ASEGURAMOS QUE NO EXISTAN DATOS FALTANTES EN EL DATAFRAME Y FILTRAMOS POR LAS COLUMNAS DE INTERÉS
mundo = mundo.dropna()
mundo = mundo.iloc[:,[9,10,11,12,13,14,5,15,16,17,18,19,20,7,8]]
mundo['CUERPOS'] = mundo['CUERPOS'].astype(int)
mundo['TOTAL CUERPOS'] = mundo['CUERPOS'] + mundo['CUERPOS ADICIONALES']

In [37]:
#mundo.to_excel(r'C:/Users/jmcastellanos/Documents/Mateo Castellanos/Proyecto Elásticidad de Espacios/Datos/SE_init_data.xlsx', index=False)